In [1]:
import ipywidgets as widgets
from IPython.display import display
import functions as functions
import importlib
import warnings
warnings.filterwarnings('ignore')


In [ ]:
import pathlib
import pandas as pd
import dropbox
from dropbox.exceptions import AuthError

DROPBOX_ACCESS_TOKEN = 'sl.BY2dvISDQFg9YN0hmkxrCtup4s6Ntn1TJlFC0X7016uDt9ms30PXI00Y8ULwo0p054eG-QBYNNDliKG1v9Jq9M68FmgYm5N1-AugJtUYa6kL_-a3dHBq_bg_GNGlIWQe2FQuTqYW6_8h'

path_log = '/eventField_test/logs'
path_config = '/eventField_test/configs'

def dropbox_connect():
    """Create a connection to Dropbox."""

    try:
        dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
    except AuthError as e:
        print('Error connecting to Dropbox with access token: ' + str(e))
    return dbx

def dropbox_list_files(path):
    """Return a Pandas dataframe of files in a given Dropbox folder path in the Apps directory.
    """

    dbx = dropbox_connect()

    try:
        files = dbx.files_list_folder(path).entries
        files_list = []
        for file in files:
            if isinstance(file, dropbox.files.FileMetadata):
                metadata = {
                    'name': file.name,
                    'path_display': file.path_display,
                    'client_modified': file.client_modified,
                    'server_modified': file.server_modified
                }
                files_list.append(metadata)

        df = pd.DataFrame.from_records(files_list)
        return df.sort_values(by='server_modified', ascending=False)

    except Exception as e:
        print('Error getting list of files from Dropbox: ' + str(e))
        

log_files = dropbox_list_files(path_log)
config_files = dropbox_list_files(path_config)

def dropbox_download_file(dropbox_file_path, local_file_path):
    """Download a file from Dropbox to the local machine."""

    try:
        dbx = dropbox_connect()

        with open(local_file_path, 'wb') as f:
            metadata, result = dbx.files_download(path=dropbox_file_path)
            f.write(result.content)
    except Exception as e:
        print('Error downloading file from Dropbox: ' + str(e))

dropbox_download_file(log_files.path_display[1], str(os.getcwd())+'/test.csv')

In [2]:
def load2(b=None):
    use_config = top_pane_dict['buttons'][0].value
    log_path = top_pane_dict['file_choosers'][0].selected
    config_path = top_pane_dict['file_choosers'][1].selected
    
    try:
        log, var_dict, startDict, raceIndex, marksDict, raceEvents, eventLog, selector_dict_day = functions.load_and_process(log_path,config_path,use_config)
        load2.log = log
        load2.var_dict = var_dict
        load2.startDict = startDict
        load2.raceIndex = raceIndex
        load2.marksDict = marksDict
        load2.raceEvents = raceEvents
        load2.eventLog = eventLog
        load2.selector_dict_day = selector_dict_day
        plots = functions.plot_race(startDict, raceEvents, log, marksDict, raceIndex) # create plots for each race
        plot_tab = functions.build_plot_tabs(plots)
        mid_pane = widgets.VBox([plot_tab,selector_dict_day['formatted']])
        load2.mid_pane = mid_pane
        
    except:
        load2.mid_pane = widgets.Label("Error Loading Files")
    



In [3]:
top_pane_dict = functions.build_top_pane()
display(top_pane_dict['formatted'][0])

In [4]:
load_button = widgets.Button(description='Load and Process Files')
mid_pane_output = widgets.Output()

    
@mid_pane_output.capture()
def load(change):
    load2()
    mid_pane_output.clear_output()
    with mid_pane_output:
        display(load2.mid_pane)
    
    
load_button.on_click(load)
display(load_button)
mid_pane_output

Button(description='Load and Process Files', style=ButtonStyle())

Output()

In [11]:
def export2(b=None):
    try:
        selector_dict_day = load2.selector_dict_day
        raceEvents = load2.raceEvents 
        startDict = load2.startDict 
        log = load2.log
        filePath_export = top_pane_dict['file_choosers'][0].selected_path
        date_fileName = str(load2.log.TimeStamp.dropna()[1].date())
        fileName_export = "eventField_"+date_fileName
        events = functions.export_to_xml_worker(selector_dict_day, raceEvents, startDict, log, filePath_export, fileName_export)
        export2.events = events
        
    
    except:
        print("Failed to export XML file")


In [12]:
export_button = widgets.Button(description='Export Events as XML')
bottom_pane_output = widgets.Output()
   
export_button.on_click(export2)
display(export_button)



Button(description='Export Events as XML', style=ButtonStyle())

Event File exported as eventField_2022-05-20
[['2022-05-20', '12:17:01', 'ManualEntry', 'Jib_batten_1=NotSet;Jib_batten_2=NotSet;Main_batten=NotSet;Forestay=0;V1=0;D1=0;D2=0;mastButt=0;=0.0'], ['2022-05-20', '12:19:01', 'SailsUp', 'MN-1;J1.5-1;;'], ['2022-05-20', '12:25:01', 'RaceStartGun', 1], ['2022-05-20', '12:38:23', 'RaceMark', 1], ['2022-05-20', '12:38:23', 'SailsUp', 'MN-1;;J1.5-1;A1.5-1'], ['2022-05-20', '12:49:42', 'RaceMark', 2], ['2022-05-20', '12:49:42', 'SailsUp', 'MN-1;J1.5-1;;'], ['2022-05-20', '13:02:37', 'RaceMark', 3], ['2022-05-20', '13:02:37', 'SailsUp', 'MN-1;;J1.5-1;A1.5-1'], ['2022-05-20', '13:11:09', 'RaceFinish', 1], ['2022-05-20', '13:11:09', 'SailsUp', 'MN-1;;;'], ['2022-05-20', '13:38:00', 'ManualEntry', 'Jib_batten_1=NotSet;Jib_batten_2=NotSet;Main_batten=NotSet;Forestay=0;V1=0;D1=0;D2=0;mastButt=0;=0.0'], ['2022-05-20', '13:40:00', 'SailsUp', 'MN-1;J1.5-1;;'], ['2022-05-20', '13:46:00', 'RaceStartGun', 2], ['2022-05-20', '14:02:17', 'RaceMark', 1], ['2022-